In [8]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [9]:
df = pd.read_excel("/content/drive/MyDrive/Scrabble/Feature ENgineering exploarion/11-23/All_metrics.xlsx")
df.rename(columns={"hist_player_score": "hist_player_performance"}, inplace=True)
df.index = df.game_id

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Drop 'game_id'
df = df.drop('game_id', axis=1)

# Set binary column based on 'first' values
df['first_bot'] = df['first'].isin(["HastyBot", "BetterBot", "STEEBot"]).astype(int)
df = df.drop('first', axis=1)

# One-hot encode categorical columns
categorical_columns = ['time_control_name', 'game_end_reason', 'lexicon', 'rating_mode', 'Bot_nickname']
df = pd.get_dummies(df, columns=categorical_columns)

# Drop 'Player_nickname' and 'created_at'
df = df.drop(['Player_nickname', 'created_at', "Bot_game"], axis=1)

# Move 'Player_rating' to the last column
cols = [col for col in df.columns if col != 'Player_rating'] + ['Player_rating']
df = df[cols]

df

,initial_time_seconds,increment_seconds,max_overtime_minutes,game_duration_seconds,Bot_turns_count,Player_turns_count,Bot_max_point,Player_max_point,Bot_min_point,Player_min_point,Bot_avg_bingo,Player_avg_bingo,Bot_word_len,Player_word_len,Bot_count_letters_JQXZ,Player_count_letters_JQXZ,Bot_rack_used_rate,Player_rack_used_rate,Bot_win,Player_win,Player_game,Bot_score,Player_score,Bot_rating,hist_player_performance,Bot_Double_Word,Bot_Triple_Word,Bot_Double_Letter,Bot_Triple_Letter,Player_Double_Word,Player_Triple_Word,Player_Double_Letter,Player_Triple_Letter,hist_player_score_20,last_game_win_flag,Player_max_point_player_wise,Player_avg_bingo_player_wise,Player_word_len_player_wise,Player_score_player_wise,hist_player_score_player_wise,Player_Double_Word_player_wise,Player_Triple_Word_player_wise,Player_Double_Letter_player_wise,Player_Triple_Letter_player_wise,hist_player_RATING_20_player_wise,first_bot,time_control_name_blitz,time_control_name_rapid,time_control_name_regular,time_control_name_ultrablitz,game_end_reason_CONSECUTIVE_ZEROES,game_end_reason_RESIGNED,game_end_reason_STANDARD,game_end_reason_TIME,lexicon_CSW21,lexicon_ECWL,lexicon_NSWL20,lexicon_NWL20,rating_mode_CASUAL,rating_mode_RATED,Bot_nickname_BetterBot,Bot_nickname_HastyBot,Bot_nickname_STEEBot,Player_rating
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
43828,15,0,1,61.405180,16,16,72,14,0,-10,0.062500,0.000000,4.187500,0.312500,2,0,0.638095,0.178571,1,0,1,472,15,2148,1700.0,4,5,8,4,0,0,1,1,1700.00,0,14,0.000000,0.312500,15,1700.0,0.000000,0.000000,1.000000,1.000000,1700.00,1,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,1979.0
5765,1200,0,1,1268.252752,11,12,92,35,16,-10,0.272727,0.000000,4.090909,2.250000,4,0,0.584416,0.482143,1,0,1,462,141,2012,1700.0,2,3,7,5,4,2,3,0,1700.00,0,35,0.026824,2.267920,141,1700.0,4.315789,3.157895,5.052632,2.631579,1700.00,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1447.0
40264,1200,0,1,1262.786704,12,12,74,36,8,-10,0.166667,0.000000,4.166667,3.000000,0,2,0.595238,0.467532,1,0,2,356,212,1966,1691.2,7,0,4,6,3,4,5,5,1695.60,0,36,0.026824,2.267920,212,1691.2,4.315789,3.157895,5.052632,2.631579,1695.60,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1408.0
44376,1200,0,1,1209.746654,15,16,102,75,8,-10,0.125000,0.062500,3.750000,2.437500,4,0,0.625397,0.396825,1,0,3,535,227,2090,1677.8,6,4,9,5,4,3,9,0,1688.90,0,75,0.026824,2.267920,227,1677.8,4.315789,3.157895,5.052632,2.631579,1688.90,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1372.0
32633,1200,0,1,1231.166683,15,16,85,85,6,-10,0.066667,0.117647,3.400000,2.823529,3,1,0.520952,0.519048,0,1,4,383,391,2114,1676.8,7,1,6,5,6,5,4,2,1688.40,0,85,0.026824,2.267920,391,1676.8,4.315789,3.157895,5.052632,2.631579,1688.40,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1367.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7264,1200,0,1,683.258274,13,14,77,78,10,0,0.076923,0.066667,3.846154,3.200000,3,1,0.549451,0.538462,1,0,7,414,327,2231,1750.7,5,2,5,4,5,5,4,1,1725.35,0,78,0.086263,3.488829,327,1750.7,5.000000,3.428571,4.142857,2.428571,1725.35,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,1,0,1855.0
66022,1380,0,1,626.274697,13,13,78,66,0,0,0.142857,0.076923,3.285714,3.846154,3,1,0.649351,0.595238,1,0,1,393,357,1859,1700.0,5,1,5,2,6,3,4,1,1700.00,0,66,0.115876,3.690110,357,1700.0,5.750000,3.250000,5.250000,3.250000,1700.00,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,1,1641.0
2581,1380,0,1,775.765387,12,11,70,76,13,17,0.076923,0.181818,3.923077,4.000000,3,1,0.642857,0.571429,0,1,2,358,438,1863,1675.9,5,2,6,5,3,4,6,5,1687.95,0,76,0.115876,3.690110,438,1675.9,5.750000,3.250000,5.250000,3.250000,1687.95,1,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,1,1637.0


In [11]:
train = df[df["Player_rating"].isnull()==False]
test = df[df["Player_rating"].isnull()==True]

In [12]:
df = train

In [13]:
df.shape

(43424, 64)

In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Activation, Dropout
from scikeras.wrappers import KerasRegressor, KerasClassifier
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [15]:
!pip install scikeras

In [17]:
X = train.drop('Player_rating', axis=1)
y = train['Player_rating']

In [18]:
warnings.filterwarnings("ignore", category=UserWarning, module="tensorflow")
# Create a pipeline with a KerasRegressor
def create_model(nb_hidden=100, activation='relu', optimizer='adam'):
    model = Sequential()
    model.add(Dense(nb_hidden, input_dim=X.shape[1], activation=activation))
    model.add(Dropout(rate=0.2))
    model.add(Dense(nb_hidden, activation=activation))
    model.add(Dropout(rate=0.2))
    model.add(Dense(nb_hidden, activation=activation))
    model.add(Dropout(rate=0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])
    return model

# Create a pipeline with a KerasRegressor
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', KerasRegressor(build_fn=create_model, nb_hidden=50, verbose=0))
])

# callbacks
checkpoint = ModelCheckpoint('Models/best_model2.h5', monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(patience=500)

# Define parameter grid for grid search
param_grid = {
    'model__nb_hidden': [50, 100, 200],
    'model__batch_size': [8, 32, 64, 256],
    'model__epochs': [12, 32],
    'model__callbacks': [[checkpoint, early_stopping]]
}
# Define RMSE as the scoring metric for GridSearchCV
rmse_scorer = make_scorer(lambda y_true, y_pred: np.sqrt(np.mean((y_true - y_pred)**2)))

# Initialize GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=10, scoring=rmse_scorer, n_jobs=-1)

# Fit the model
grid_search.fit(X, y)

# Get the best parameters
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Get the best model
best_model = grid_search.best_estimator_
print("Best Model:", best_model)

# Get cross-validation scores
print("Running cross-validation. This may take some time...")
kf = KFold(n_splits=10, shuffle=True, random_state=42)  # Specify the same number of splits as the GridSearchCV
fold = 1
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)

    fold_rmse = np.sqrt(np.mean((y_test - y_pred)**2))
    print(f"Fold {fold}: RMSE = {fold_rmse}")
    fold += 1

warnings.resetwarnings()

Best Parameters: {'model__batch_size': 256, 'model__callbacks': [<keras.src.callbacks.ModelCheckpoint object at 0x7cc3119e1a50>, <keras.src.callbacks.EarlyStopping object at 0x7cc3119e1030>], 'model__epochs': 12, 'model__nb_hidden': 50}
Best Model: Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 KerasRegressor(batch_size=256, build_fn=<function create_model at 0x7cc37b1fecb0>, callbacks=[<keras.src.callbacks.ModelCheckpoint object at 0x7cc37a225510>, <keras.src.callbacks.EarlyStopping object at 0x7cc37a227b80>], epochs=12, nb_hidden=50, verbose=0))])
Running cross-validation. This may take some time...


Fold 1: RMSE = 119.35602004977798


Fold 2: RMSE = 117.65337941579384


Fold 3: RMSE = 134.5028595569429


Fold 4: RMSE = 123.66606505566963


...

Fold 6: RMSE = 122.35657470638739


Fold 7: RMSE = 119.55547602757717


Fold 8: RMSE = 129.18043279876483


Fold 9: RMSE = 123.83112980816988


Fold 10: RMSE = 121.25506349589665


In [19]:
best_model

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 KerasRegressor(batch_size=256, build_fn=<function create_model at 0x7cc37b1fecb0>, callbacks=[<keras.src.callbacks.ModelCheckpoint object at 0x7cc37a225510>, <keras.src.callbacks.EarlyStopping object at 0x7cc37a227b80>], epochs=12, nb_hidden=50, verbose=0))])